# Pull data from Hybrid-Analysis using their API and make a model to predict whether traffic is malicious or no

### conda install requests

In [1]:
import requests
import json

### Set headers for the API request

In [2]:
parameters = {
    "api-key": "s3vwbiju16f29800tnrm7m1o11d8af609qpen4wg539481160qg3q7rh3c96aabc",
    "accept-encoding" : "gzip",
    "user-agent" : "Falcon Sandbox"
}

### Pull from feed

In [3]:
response = requests.get("https://www.hybrid-analysis.com/feed?json", headers=parameters)
## log out the return code and any headers - not required for this problem but useful
print("response code: ", response.status_code, "content length is ", len(response.content))
print("headers: ", json.dumps(response.headers.__dict__, sort_keys=True, indent=4))

response code:  200 content length is  3248401
headers:  {
    "_store": {
        "access-control-allow-origin": [
            "Access-Control-Allow-Origin",
            "*"
        ],
        "cache-control": [
            "Cache-Control",
            "max-age=0, private, must-revalidate, max-age=2592000"
        ],
        "cf-cache-status": [
            "CF-Cache-Status",
            "DYNAMIC"
        ],
        "cf-ray": [
            "CF-RAY",
            "63be062cb8f52752-DEN"
        ],
        "cf-request-id": [
            "cf-request-id",
            "094aa22ff50000275299258000000001"
        ],
        "connection": [
            "Connection",
            "keep-alive"
        ],
        "content-encoding": [
            "Content-Encoding",
            "gzip"
        ],
        "content-security-policy": [
            "Content-Security-Policy",
            "default-src 'none'; connect-src 'self' *.twitter.com; script-src 'self' *.google.com *.gstatic.com *.google-analytics.

In [4]:
json_response = response.json()
datasets = json_response['data']
print(datasets)

[{'md5': 'faa3e8b8210b46a2c33a079b58bcc153', 'sha1': '6b8e3e9758048267e7b79b70f364874685b6d45b', 'sha256': '1101a544f5329a865cfdb9850d8e20a7dd69d94854fdde1a4c964c7421b1f5c3', 'isunknown': True, 'isinteresting': False, 'analysis_start_time': '2021-04-06 21:02:49', 'threatscore': 20, 'threatlevel': 0, 'threatlevel_human': 'no specific threat', 'submitname': 'http://sp.cwfservice.net/2/r/f65827d5d3a25148b8d0fc63312be800/brdwsep001/0/-/https/freychang.fun/443/', 'isurlanalysis': True, 'domains': ['cacerts.digicert.com', 'sp.cwfservice.net'], 'hosts': ['168.149.132.176', '184.84.68.145', '104.18.11.39', '104.18.10.39'], 'hosts_geo': [{'ip': '168.149.132.176', 'lat': '-23.5475', 'lon': '-46.6361', 'cc': 'USA'}, {'ip': '184.84.68.145', 'lat': '39.0437', 'lon': '-77.4875', 'cc': 'USA'}, {'ip': '104.18.11.39', 'lat': '37.7621', 'lon': '-122.3971', 'cc': 'USA'}, {'ip': '104.18.10.39', 'lat': '37.7621', 'lon': '-122.3971', 'cc': 'USA'}], 'environmentId': '100', 'environmentDescription': 'Windows 

In [5]:
json_response = response.json()
datasets = json_response['data']
print (len(datasets))
print ("first one is of type ", type(datasets[0]))
#print ("first one is ", datasets[0])
print (datasets[0].keys())
print()
#print ("first one is ", json.dumps(datasets[0], sort_keys=True, indent=4))
print ("is interesting: ", datasets[0].get("isinteresting"), " threatscore:", datasets[0].get("threatscore"), "reporturl: ", datasets[0].get("reporturl"), " sha256:", datasets[0].get("sha256"))

201
first one is of type  <class 'dict'>
dict_keys(['md5', 'sha1', 'sha256', 'isunknown', 'isinteresting', 'analysis_start_time', 'threatscore', 'threatlevel', 'threatlevel_human', 'submitname', 'isurlanalysis', 'domains', 'hosts', 'hosts_geo', 'environmentId', 'environmentDescription', 'sharedanalysis', 'isreliable', 'reporturl', 'process_list', 'extracted_files', 'ssdeep'])

is interesting:  False  threatscore: 20 reporturl:  /sample/1101a544f5329a865cfdb9850d8e20a7dd69d94854fdde1a4c964c7421b1f5c3/606ccc728382bd3df42ff63b  sha256: 1101a544f5329a865cfdb9850d8e20a7dd69d94854fdde1a4c964c7421b1f5c3


### This for loop is just practice for a similar for loop later on. Not used in model

In [46]:
for aThing in datasets:
    print ("is interesting: ", aThing.get("isinteresting"), " threatscore:", aThing.get("threatscore"),  " reporturl:", aThing.get("reporturl"),  " sha256:", aThing.get("sha256"))

is interesting:  False  threatscore: 58  reporturl: /sample/0a153691c2f0c38c5cb19dab03309be43a3484382260764c1279e0793fe60cc3/606b3697c7e97850b612ad68  sha256: 0a153691c2f0c38c5cb19dab03309be43a3484382260764c1279e0793fe60cc3
is interesting:  False  threatscore: 0  reporturl: /sample/3b4b5852acbc869ec3dd96a116417db41426bb1b0dfc83915043a25f2631c724/606b36727391dd62ad5850ac  sha256: 3b4b5852acbc869ec3dd96a116417db41426bb1b0dfc83915043a25f2631c724
is interesting:  False  threatscore: 28  reporturl: /sample/42f5e9f03d0fd13e8077bb3a22f22bd8b7c0d2184ce2f853ca9220401210adbf/606b361bd808da34ae321a3b  sha256: 42f5e9f03d0fd13e8077bb3a22f22bd8b7c0d2184ce2f853ca9220401210adbf
is interesting:  False  threatscore: 23  reporturl: /sample/891b7b2688fd3c3db28fe3d6afe2d0f05680cc5e5819293d443eb436515acd10/606b361a0de17b7554787104  sha256: 891b7b2688fd3c3db28fe3d6afe2d0f05680cc5e5819293d443eb436515acd10
is interesting:  False  threatscore: 30  reporturl: /sample/3f658cd4bddd86c72be04adf671609265971e26250e8c

### Format the data into a dataframe

In [6]:
import pandas as pd
df = pd.DataFrame(datasets)
df

,md5,sha1,sha256,isunknown,isinteresting,analysis_start_time,threatscore,threatlevel,threatlevel_human,submitname,...,tags,size,type,et_alerts_total,et_alerts_real_total,et_alerts,domains_capped,hosts_capped,vxfamily,associated_email_headers
0,faa3e8b8210b46a2c33a079b58bcc153,6b8e3e9758048267e7b79b70f364874685b6d45b,1101a544f5329a865cfdb9850d8e20a7dd69d94854fdde...,True,False,2021-04-06 21:02:49,20,0,no specific threat,http://sp.cwfservice.net/2/r/f65827d5d3a25148b...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01a33301798a6f3baf52a5abddb36295,5cf1cbae63ecfe54652efa737647fdcce52e99fd,1ae2b7b8a63b858a50f4cebebc46b2b81fc5c7af377571...,False,False,2021-04-06 21:02:19,23,0,no specific threat,https://pegalgerie.dz/gfgfth/Drive,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,bec44c0d732f81206f157563e0a90f0e,78d277e56ae4fa6355ebb5b1502844f85005ab52,0be434aad3fd6c1ebd808a3ea1508377cc6941d2539def...,True,True,2021-04-06 21:00:10,85,2,malicious,1617742801_w7x32_pafish.exe,...,[evasive],105386.0,PE32 executable (console) Intel 80386 (strippe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,22f7c3c2c4736e57182121f106924476,20c1b564b81e57678420f289aacf78a94cc237c8,e72fcec507f8531e3c29e846035641df93fd35433a25fe...,True,True,2021-04-06 21:00:10,90,2,malicious,1617742802_pafish.exe,...,[evasive],105386.0,PE32 executable (console) Intel 80386 (strippe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ab4e3b68767e61df64826c2525a81448,0b7ef7fa650e5aeb25e2d96fb9effc3670f18805,62c1ad03d51e01726b89f3c5d2fe226d9b867e8ae07645...,True,False,2021-04-06 21:00:10,26,0,no specific threat,https://google.com/search?q=NjliZWNlMTBmYWEzMT...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,71f01259a445a01476551641a2ce3825,137bb9ff9309e0a69f8443cdc83f8d4a6756ede7,a5137c26a9610cdadad10fd14cc8efdc164359a3827f27...,True,False,2021-04-06 20:03:27,13,0,no specific threat,https://www.vlive.tv/post/0-22189519,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,d53e9a3edb77a1405dd6ef8029a93784,b04bf69d7b7c06580eee033653b522e9a78b41d1,d05dea15ed5852c4d5aed1607cfd42ef0971ef4f13d683...,True,False,2021-04-06 20:03:07,13,0,no specific threat,https://www.vlive.tv/post/0-22189516,...,NaN,NaN,NaN,9.0,9.0,"[{'destip': '104.97.222.254', 'destport': '80'...",NaN,NaN,NaN,NaN
198,07f1c12fdd6798d11cf7403935e47834,8106c0b710902d651453d2b8dd9b06556d3ff6fc,da4cb53218212bdddbaad8095a38f2b41be2818a64a268...,True,False,2021-04-06 20:03:06,23,0,no specific threat,https://www.open.edu/openlearn/node/1787849/fu...,...,NaN,NaN,NaN,9.0,9.0,"[{'destip': '104.117.148.96', 'destport': '80'...",NaN,NaN,NaN,NaN
199,936175a3079f62c1011d5dad7946da9b,6e0c89f6fd6699350ca8b43117ca3af33b470649,088d85ae34dd1c509a973c56f48f4148910e8518cd81f4...,True,False,2021-04-06 20:02:16,21,0,no specific threat,6sbcvx-pokemon-go-generator.pdf,...,NaN,304645.0,"PDF document, version 1.3",NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Look at the reporturl column to see how we need to split it up so we can put the values in a for loop to pull each individual report

In [7]:
df["reporturl"]

0      /sample/1101a544f5329a865cfdb9850d8e20a7dd69d9...
1      /sample/1ae2b7b8a63b858a50f4cebebc46b2b81fc5c7...
2      /sample/0be434aad3fd6c1ebd808a3ea1508377cc6941...
3      /sample/e72fcec507f8531e3c29e846035641df93fd35...
4      /sample/62c1ad03d51e01726b89f3c5d2fe226d9b867e...
                             ...                        
196    /sample/a5137c26a9610cdadad10fd14cc8efdc164359...
197    /sample/d05dea15ed5852c4d5aed1607cfd42ef0971ef...
198    /sample/da4cb53218212bdddbaad8095a38f2b41be281...
199    /sample/088d85ae34dd1c509a973c56f48f4148910e85...
200    /sample/6b60d170a8381739593b40ffa720b346cf63c0...
Name: reporturl, Length: 201, dtype: object

### Split the report url to seperate out jobid

In [8]:
#Filter out /sample/ from reporturl column
df["jobid"]=df["reporturl"].str.split("/", expand=True)[3]
list(df)

['md5',
 'sha1',
 'sha256',
 'isunknown',
 'isinteresting',
 'analysis_start_time',
 'threatscore',
 'threatlevel',
 'threatlevel_human',
 'submitname',
 'isurlanalysis',
 'domains',
 'hosts',
 'hosts_geo',
 'environmentId',
 'environmentDescription',
 'sharedanalysis',
 'isreliable',
 'reporturl',
 'process_list',
 'extracted_files',
 'ssdeep',
 'avdetect',
 'compromised_hosts',
 'vt_detect',
 'ms_detect',
 'tags',
 'size',
 'type',
 'et_alerts_total',
 'et_alerts_real_total',
 'et_alerts',
 'domains_capped',
 'hosts_capped',
 'vxfamily',
 'associated_email_headers',
 'jobid']

### Combine url with job id so we can insert into for loop to pull all the matching reports from the feed.

In [9]:
df["url"]= "https://www.hybrid-analysis.com/api/v2/report/"+df["jobid"]+"/summary"
df["url"]

0      https://www.hybrid-analysis.com/api/v2/report/...
1      https://www.hybrid-analysis.com/api/v2/report/...
2      https://www.hybrid-analysis.com/api/v2/report/...
3      https://www.hybrid-analysis.com/api/v2/report/...
4      https://www.hybrid-analysis.com/api/v2/report/...
                             ...                        
196    https://www.hybrid-analysis.com/api/v2/report/...
197    https://www.hybrid-analysis.com/api/v2/report/...
198    https://www.hybrid-analysis.com/api/v2/report/...
199    https://www.hybrid-analysis.com/api/v2/report/...
200    https://www.hybrid-analysis.com/api/v2/report/...
Name: url, Length: 201, dtype: object

### DO NOT USE THIS ONE: Use api to pull report summary data 

In [51]:
#for index, row in df.iterrows():
    ## log out the return code and any headers - not required for this problem but useful
    #print(row["url"])
    #response = requests.get(row["url"], headers=parameters)
    #print("response code: ", response.status_code, "content length is ", len(response.content))
    #print("headers: ", json.dumps(response.headers.__dict__, sort_keys=True, indent=4))
    #json_response = response.json()
    #report = json_response['data']
    #print(json_response)
    
    #df.at[index,"summary"] = json_response

### This call puts the json data and puts it into a new "summary" column

In [10]:
def get_summary(row):
    #print(row)
    response = requests.get(row["url"], headers=parameters)
    print("response code: ", response.status_code, "content length is ", len(response.content))
    #print("headers: ", json.dumps(response.headers.__dict__, sort_keys=True, indent=4))
    json_response = response.json()
    #report = json_response['data']
    #print(json_response)
    
    return json_response
    
df["summary"] = df.apply(get_summary, axis = 1)

response code:  200 content length is  27495
response code:  200 content length is  29125
response code:  200 content length is  6891
response code:  200 content length is  7603
response code:  200 content length is  12510
response code:  200 content length is  22987
response code:  200 content length is  15286
response code:  200 content length is  32541
response code:  200 content length is  10262
response code:  200 content length is  36321
response code:  200 content length is  33791
response code:  200 content length is  10189
response code:  200 content length is  4104
response code:  200 content length is  14852
response code:  200 content length is  14517
response code:  200 content length is  6860
response code:  200 content length is  31310
response code:  200 content length is  7634
response code:  200 content length is  29856
response code:  200 content length is  11480
response code:  200 content length is  26382
response code:  200 content length is  32454
response code: 

### Make sure it worked and added the new summary column

In [11]:
df["summary"]

0      {'job_id': '606ccc728382bd3df42ff63b', 'enviro...
1      {'job_id': '606ccc5582eb4e39336d3daf', 'enviro...
2      {'job_id': '606ccbd398da4b65ca4ec189', 'enviro...
3      {'job_id': '606ccbd36f7cae47cb3bf964', 'enviro...
4      {'job_id': '606ccbd30cd5b96b8c5b20c0', 'enviro...
                             ...                        
196    {'job_id': '606cbe870c503a478c3c2f0b', 'enviro...
197    {'job_id': '606cbe73216b6e1ceb2a69c9', 'enviro...
198    {'job_id': '606cbe750098b96374036374', 'enviro...
199    {'job_id': '606cbe436d35190c967a7792', 'enviro...
200    {'code': 429, 'message': 'Exceeded maximum API...
Name: summary, Length: 201, dtype: object

### Just checking the summary column type

In [12]:
print(type(df["summary"]))
print(df["summary"].dtypes)

<class 'pandas.core.series.Series'>
object


### This makes sure no failed reports are included. Aka if we run out of daily requests those aren't added into the dataframe. The block below shows the length of each summary.

In [13]:
df['sum_len'] = df['summary'].str.len()
print(df['sum_len'])

0      41
1      41
2      41
3      41
4      41
       ..
196    41
197    41
198    41
199    41
200     2
Name: sum_len, Length: 201, dtype: int64


### This block gets rid of all the summary's that didn't come through. their length is less than 2

In [14]:
df = df[df.sum_len > 2]
print(df['sum_len'])

0      41
1      41
2      41
3      41
4      41
       ..
195    41
196    41
197    41
198    41
199    41
Name: sum_len, Length: 200, dtype: int64


### Create function to make a new row counting the number of mitre attacks

In [15]:
#for index, row in df.iterrows():
    # log out the return code and any headers - not required for this problem but useful
    #print(len(row["summary"]["mitre_attcks"]))
    
def get_num_mitre_attcks(row):
    return len(row["summary"]["mitre_attcks"])

    
df["num_mitre_attcks"] = df.apply(get_num_mitre_attcks, axis = 1)
print(df['num_mitre_attcks'])

0       0
1       0
2      14
3      16
4       0
       ..
195     1
196     0
197     0
198     0
199     0
Name: num_mitre_attcks, Length: 200, dtype: int64


<ipython-input-15-4d66e01f2eb4>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["num_mitre_attcks"] = df.apply(get_num_mitre_attcks, axis = 1)


# Model Building
### Import needed packages

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


%matplotlib inline
plt.style.use('ggplot')
import warnings; warnings.simplefilter('ignore')

# Make a copy of df. We can only access so many pulls in one day from the api so it is imperitive to work "offline" to keep testing

In [17]:
df2 = df.copy()

### Make the 'isinteresting' column a binary

In [18]:
df2 = df2.reset_index()
np.isnan(df2.any()) #and gets False
np.isfinite(df2.all()) #and gets True
df2

,index,md5,sha1,sha256,isunknown,isinteresting,analysis_start_time,threatscore,threatlevel,threatlevel_human,...,et_alerts,domains_capped,hosts_capped,vxfamily,associated_email_headers,jobid,url,summary,sum_len,num_mitre_attcks
0,0,faa3e8b8210b46a2c33a079b58bcc153,6b8e3e9758048267e7b79b70f364874685b6d45b,1101a544f5329a865cfdb9850d8e20a7dd69d94854fdde...,True,False,2021-04-06 21:02:49,20,0,no specific threat,...,NaN,NaN,NaN,NaN,NaN,606ccc728382bd3df42ff63b,https://www.hybrid-analysis.com/api/v2/report/...,"{'job_id': '606ccc728382bd3df42ff63b', 'enviro...",41,0
1,1,01a33301798a6f3baf52a5abddb36295,5cf1cbae63ecfe54652efa737647fdcce52e99fd,1ae2b7b8a63b858a50f4cebebc46b2b81fc5c7af377571...,False,False,2021-04-06 21:02:19,23,0,no specific threat,...,NaN,NaN,NaN,NaN,NaN,606ccc5582eb4e39336d3daf,https://www.hybrid-analysis.com/api/v2/report/...,"{'job_id': '606ccc5582eb4e39336d3daf', 'enviro...",41,0
2,2,bec44c0d732f81206f157563e0a90f0e,78d277e56ae4fa6355ebb5b1502844f85005ab52,0be434aad3fd6c1ebd808a3ea1508377cc6941d2539def...,True,True,2021-04-06 21:00:10,85,2,malicious,...,NaN,NaN,NaN,NaN,NaN,606ccbd398da4b65ca4ec189,https://www.hybrid-analysis.com/api/v2/report/...,"{'job_id': '606ccbd398da4b65ca4ec189', 'enviro...",41,14
3,3,22f7c3c2c4736e57182121f106924476,20c1b564b81e57678420f289aacf78a94cc237c8,e72fcec507f8531e3c29e846035641df93fd35433a25fe...,True,True,2021-04-06 21:00:10,90,2,malicious,...,NaN,NaN,NaN,NaN,NaN,606ccbd36f7cae47cb3bf964,https://www.hybrid-analysis.com/api/v2/report/...,"{'job_id': '606ccbd36f7cae47cb3bf964', 'enviro...",41,16
4,4,ab4e3b68767e61df64826c2525a81448,0b7ef7fa650e5aeb25e2d96fb9effc3670f18805,62c1ad03d51e01726b89f3c5d2fe226d9b867e8ae07645...,True,False,2021-04-06 21:00:10,26,0,no specific threat,...,NaN,NaN,NaN,NaN,NaN,606ccbd30cd5b96b8c5b20c0,https://www.hybrid-analysis.com/api/v2/report/...,"{'job_id': '606ccbd30cd5b96b8c5b20c0', 'enviro...",41,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,195,517901ebc9ee5099d700471e1d8f0041,7fe323cc52999fe5a799a785f771c7b1cbe9aec4,073494b8671257cea7d7a133807ff64140f62a5ebf9471...,False,False,2021-04-06 20:03:36,47,1,suspicious,...,NaN,NaN,NaN,NaN,NaN,606cbe961158ce441e1f9e86,https://www.hybrid-analysis.com/api/v2/report/...,"{'job_id': '606cbe961158ce441e1f9e86', 'enviro...",41,1
196,196,71f01259a445a01476551641a2ce3825,137bb9ff9309e0a69f8443cdc83f8d4a6756ede7,a5137c26a9610cdadad10fd14cc8efdc164359a3827f27...,True,False,2021-04-06 20:03:27,13,0,no specific threat,...,NaN,NaN,NaN,NaN,NaN,606cbe870c503a478c3c2f0b,https://www.hybrid-analysis.com/api/v2/report/...,"{'job_id': '606cbe870c503a478c3c2f0b', 'enviro...",41,0
197,197,d53e9a3edb77a1405dd6ef8029a93784,b04bf69d7b7c06580eee033653b522e9a78b41d1,d05dea15ed5852c4d5aed1607cfd42ef0971ef4f13d683...,True,False,2021-04-06 20:03:07,13,0,no specific threat,...,"[{'destip': '104.97.222.254', 'destport': '80'...",NaN,NaN,NaN,NaN,606cbe73216b6e1ceb2a69c9,https://www.hybrid-analysis.com/api/v2/report/...,"{'job_id': '606cbe73216b6e1ceb2a69c9', 'enviro...",41,0
198,198,07f1c12fdd6798d11cf7403935e47834,8106c0b710902d651453d2b8dd9b06556d3ff6fc,da4cb53218212bdddbaad8095a38f2b41be2818a64a268...,True,False,2021-04-06 20:03:06,23,0,no specific threat,...,"[{'destip': '104.117.148.96', 'destport': '80'...",NaN,NaN,NaN,NaN,606cbe750098b96374036374,https://www.hybrid-analysis.com/api/v2/report/...,"{'job_id': '606cbe750098b96374036374', 'enviro...",41,0


### Check how many interesting rows there are

In [29]:
df2["isinteresting_binary"] = df2["isinteresting"].astype(int)

Total = df2['isinteresting_binary'].sum()
print (Total)

26


### Try making nan values 0

In [30]:
def clean_dataset(df2):
    assert isinstance(df2, pd.DataFrame), "df needs to be a pd.DataFrame"
    df2.dropna(inplace=True)
    indices_to_keep = ~df2.isin([np.nan, np.inf, -np.inf]).any(1)
    return df2[indices_to_keep].astype(np.float64)

### Add input variables and split the testing data on a 70/30 split

In [31]:
X=df2[['threatscore', 'threatlevel', 'num_mitre_attcks']]  # Features
y=df2['isinteresting_binary']  # Labels

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

### Random Forest model. Create a classifier and fit on training data

In [32]:
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
cla=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
cla.fit(X_train,y_train)

# prediction on test set
y_pred=cla.predict(X_test)

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


Accuracy: 0.9666666666666667
